In [644]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [645]:
secrets_resolver = SecretsResolverDemo()

In [646]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
             didcomm_services = []
#                 DIDCommService(
#                     id = s.id,
#                     service_endpoint = s.service_endpoint,
#                     routing_keys = s.routing_keys,
#                     accept = s.accept
#                 )
#                 for s in did_doc.service
#                 if isinstance(s, DIDCommServicePeerDID)
#            ] if did_doc.service else []
        )

In [711]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, 
                    routing_keys=[],
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Alice get OOB from Mediator

In [649]:
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [650]:
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiNzk0Mjc4MzctY2MwNi00ODUzLWJiMzktNjg2ZWFjM2U2YjlhIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNtczU1NVloRnRobjFXVjhjaURCcFptODZoSzl0cDgzV29qSlVteFBHazFoWi5WejZNa21kQmpNeUI0VFM1VWJiUXc1NHN6bTh5dk1NZjFmdEdWMnNRVllBeGFlV2hFLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwibGFiZWwiOiJNZWRpYXRvciIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ


In [651]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)
print(received_msg_decoded)

Alice's DID: did:peer:2.Ez6LSnPMs71uWDN6UY9pVjoBEWS9pP6SSxoQAZnvnRn4bpH6y.Vz6MkrBgzPPvHc6PkAc61GYZU6yyGtagvnLGAgvvXZ3ve3pbz
{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '79427837-cc06-4853-bb39-686eac3e6b9a', 'from': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'label': 'Mediator', 'accept': ['didcomm/v2']}}


### Alice request mediate

In [652]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [653]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [654]:

mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_did_doc)

{'id': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'publicKeyMultibase': 'z6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftG

In [655]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, json = json.loads(alice_mediate_grant_packed.packed_msg))


In [656]:
print(resp.json())

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTIxd01tVjVja0pYUkRaa016Vk5aRTB4ZG1WSVNHRkVOVGh2TVdoelV6WlJPVkpaYlZCaWIxVllZVTVCTGxaNk5rMXJjREpTV0d0bFNuUTBRa2g2VVZGMlkwUjFPRzVSUnpoemNIQnFURkJVUkRodWMweFhha3REWVRORVRIRXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTIxd01tVjVja0pYUkRaa016Vk5aRTB4ZG1WSVNHRkVOVGh2TVdoelV6WlJPVkpaYlZCaWIxVllZVTVCIiwiYXB2IjoiV000S1FKYThVcUR0ZWtTS1J5dXd3UnhZY0pDSGtZUDFUU2hRQUlxbWlyMCIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTbXAyZXlyQldENmQzNU1kTTF2ZUhIYUQ1OG8xaHNTNlE5UlltUGJvVVhhTkEuVno2TWtwMlJYa2VKdDRCSHpRUXZjRHU4blFHOHNwcGpMUFREOG5zTFdqS0NhM0RMcS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTb

### Get mediator grant response

In [657]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

https://didcomm.org/coordinate-mediation/2.0/mediate-grant
Routing Key: did:peer:2.Ez6LSjTa6gDXWTy9aPzaaqzisYexethq2htD6PPaTd2R9Zdeo.Vz6MkgejsAq3w3s7fSPjmPQ4JmDUGottEmR7MxBqcWyesrrwM.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0
Mediator new DID: did:peer:2.Ez6LSmp2eyrBWD6d35MdM1veHHaD58o1hsS6Q9RYmPboUXaNA.Vz6Mkp2RXkeJt4BHzQQvcDu8nQG8sppjLPTD8nsLWjKCa3DLq.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [712]:
#alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
#alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=[{"uri": mediator_routing_key}])
alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=mediator_routing_key)


print("Alice's DID for Bob:", alice_did_for_bob)

Alice's DID for Bob: did:peer:2.Ez6LSd1naudB3T2PYA4G5NtkZaEiQKZMdZXFxHEGUECoWahGt.Vz6MkqXrYaNZAa5dtLi2wurXcDU9khMmTQGXed8RT17rTRwyt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTalRhNmdEWFdUeTlhUHphYXF6aXNZZXhldGhxMmh0RDZQUGFUZDJSOVpkZW8uVno2TWtnZWpzQXEzdzNzN2ZTUGptUFE0Sm1EVUdvdHRFbVI3TXhCcWNXeWVzcnJ3TS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0


Alice create and send the keylist update message

In [713]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_for_bob,
            "action": "add"
        }
    ]
    }
)

In [714]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [715]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [716]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSd1naudB3T2PYA4G5NtkZaEiQKZMdZXFxHEGUECoWahGt.Vz6MkqXrYaNZAa5dtLi2wurXcDU9khMmTQGXed8RT17rTRwyt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTalRhNmdEWFdUeTlhUHphYXF6aXNZZXhldGhxMmh0RDZQUGFUZDJSOVpkZW8uVno2TWtnZWpzQXEzdzNzN2ZTUGptUFE0Sm1EVUdvdHRFbVI3TXhCcWNXeWVzcnJ3TS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'action': 'add', 'result': 'success'}]}


### Alice can now send her DID to Bob.

### Alice check status of queued messages from the Mediator

In [361]:
alice_status_check = Message(
    id = "unique-id-293e9a922efffxxx",
    type = "https://didcomm.org/messagepickup/3.0/status-request",
    body = {}
)

In [362]:
alice_status_check_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_status_check,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)

In [363]:
mediator_unpack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp3.json()
)
print(mediator_unpack_status.message.type)
print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))

https://didcomm.org/messagepickup/3.0/status
Messages in Mediator queue: 1


### If there are messages, Alice can pickup from the mediator (delivery request)

In [364]:
alice_delivery_request = Message(
    id = "unique-id-293e9a922efffxxxff",
    type = "https://didcomm.org/messagepickup/3.0/delivery-request",
    body = {"limit": 1}
)

In [365]:
alice_delivery_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_delivery_request,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)

In [366]:
mediator_delivery = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp4.json()
)
print(mediator_delivery.message.type)
print(mediator_delivery.message.body)

https://didcomm.org/messagepickup/3.0/delivery
{'recipient_key': None}


In [367]:
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

In [368]:
print(bob_packed_msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNSTFpGZFhSVEpWUTBWcFMxWlNZVXhoUkRWT1JsaGhhRVJIWjFreGQzbHhaVFZUVUVSTVZGbE1PV2hCTGxaNk5rMXJaak5FVlhjME1YWmxkMmxGUmpWSFJGcDZkazF1TVc1aU1WRm9ZMUpLZUZKWmJqY3hZbmswYW5ReVdqWXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3elpETmtlVFZzWlVkR2RHTkhlR3hNYlU1MllsTTVhV0l5U1dsTVEwcG9TV3B3WWtsdFVuQmFSMDUyWWxjd2RtUnFTV2xZV0RBak5reFRkRXRrVjFkRk1sVkRSV2xMVmxKaFRHRkVOVTVHV0dGb1JFZG5XVEYzZVhGbE5WTlFSRXhVV1V3NWFFRSIsImFwdiI6IlJzZ3d4Zm5jbndGQXBmekNrbGdIYWFodFItRjJwSDRwS2RNRVNVWFE5ZWsiLCJza2lkIjoiZGlkOnBlZXI6Mi5FejZMU3RLZFdXRTJVQ0VpS1ZSYUxhRDVORlhhaERHZ1kxd3lxZTVTUERMVFlMOWhBLlZ6Nk1rZjNEVXc0MXZld2lFRjVHRFp6dk1uMW5iMVFoY1JKeFJZbjcxYnk0anQyWjYuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5aWIySWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTdEtkV1dFMlVDR

In [369]:
bob_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= bob_packed_msg
)

In [370]:
print("Message ID:", msg_id)
print(bob_msg.message.body["content"])

Message ID: 63972feb6eee089c8c00a11a
Argentina or Croacia at World Cup finals?


### Alice acknowledge so the mediator can delete the message

In [371]:
alice_ack = Message(
    id = "unique-id-293e9a922efffxxxffsss",
    type = "https://didcomm.org/messagepickup/3.0/messages-received",
    body = {"message_id_list": [msg_id]}
)

In [372]:
alice_ack_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_ack,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)

In [373]:
mediator_ack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp5.json()
)
print(mediator_ack_status.message.type)
print(mediator_ack_status.message.body)

https://didcomm.org/messagepickup/3.0/status
{'message_count': 0, 'live_delivery': False}


## Alice receive invitation from Prism Agent

In [717]:
oob_url_prism = "https://domain.com/path?_oob=eyJpZCI6ImI2OTYyNzFjLTk3MjMtNDA5Ni04ZmNiLTgyN2VkNTRmYTliNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNmWDlYZWVVclhDaEc1eXpXdjJ4ZVBUa0pDdlc4aWJWeTZNd21qcW9jb2d4WS5WejZNa3RkcUdXcUdBejhkMlU1aWluRGRQM3lrY05jeFFqcU1XZnJ1RlJKaDZhZUU3LlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92THpFNU1pNHhOamd1T0RZdU5UTTZPREE0TUM5a2FXUmpiMjF0SWl3aWNpSTZXMTBzSW1FaU9sc2laR2xrWTI5dGJTOTJNaUpkZlEiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ=="

In [718]:
received_msg_encoded = oob_url_prism.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'id': 'b696271c-9723-4096-8fcb-827ed54fa9b4', 'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'from': 'did:peer:2.Ez6LSfX9XeeUrXChG5yzWv2xePTkJCvW8ibVy6MwmjqocogxY.Vz6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'body': {'goal_code': 'connect', 'goal': 'Establish a trust connection between two peers', 'accept': []}}


In [719]:
prism_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
prism_endpoint = prism_did_doc["service"][0]["serviceEndpoint"]
prism_did = received_msg_decoded["from"]
print(prism_did_doc)
print(prism_endpoint)
print(prism_did)

{'id': 'did:peer:2.Ez6LSfX9XeeUrXChG5yzWv2xePTkJCvW8ibVy6MwmjqocogxY.Vz6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'authentication': [{'id': 'did:peer:2.Ez6LSfX9XeeUrXChG5yzWv2xePTkJCvW8ibVy6MwmjqocogxY.Vz6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ#6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSfX9XeeUrXChG5yzWv2xePTkJCvW8ibVy6MwmjqocogxY.Vz6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'publicKeyMultibase': 'z6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSfX9XeeUrXChG5yzWv2xePTkJCvW8ibVy6MwmjqocogxY.Vz6MktdqGWqGAz8d2U5iinDdP3ykcNcxQjqMWfruFRJh6aeE7.S

In [720]:
#alice_did_for_prism=await create_peer_did(1, 1, service_endpoint="http://host.docker.internal:8090/didcomm/")
print(alice_did_for_bob)
print(json.loads(peer_did.resolve_peer_did(alice_did_for_bob)))

did:peer:2.Ez6LSd1naudB3T2PYA4G5NtkZaEiQKZMdZXFxHEGUECoWahGt.Vz6MkqXrYaNZAa5dtLi2wurXcDU9khMmTQGXed8RT17rTRwyt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTalRhNmdEWFdUeTlhUHphYXF6aXNZZXhldGhxMmh0RDZQUGFUZDJSOVpkZW8uVno2TWtnZWpzQXEzdzNzN2ZTUGptUFE0Sm1EVUdvdHRFbVI3TXhCcWNXeWVzcnJ3TS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0
{'id': 'did:peer:2.Ez6LSd1naudB3T2PYA4G5NtkZaEiQKZMdZXFxHEGUECoWahGt.Vz6MkqXrYaNZAa5dtLi2wurXcDU9khMmTQGXed8RT17rTRwyt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJkaWQ6cGVlcjoyLkV6NkxTalRhNmdEWFdUeTlhUHphYXF6aXNZZXhldGhxMmh0RDZQUGFUZDJSOVpkZW8uVno2TWtnZWpzQXEzdzNzN2ZTUGptUFE0Sm1EVUdvdHRFbVI3TXhCcWNXeWVzcnJ3TS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LSd1n

In [721]:
connection_request = Message(
    id = "46430433-d872-4d4c-8376-0fa72cf124c2",
    thid = "aaa456f3-8c51-4b1c-91e4-87e51c7a5104",
    type = "https://atalaprism.io/mercury/connections/1.0/request",
    body = {"goal_code": "Connect","goal":"Connect","accept":["didcommv2"]},
    frm = alice_did_for_bob, 
    to = [prism_did]
)

In [722]:
connection_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = connection_request,
    frm = alice_did_for_bob,
    to = prism_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [723]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post("http://127.0.0.1:8080/didcomm", headers=headers, data = connection_request_packed_msg.packed_msg)

In [724]:
print(resp)

<Response [500]>


In [725]:
testDID = "did:peer:2.Ez6LSdGpdLzSijvKbyLmCf5n6o7cWi49kkUQB697hAp2GSNdu.Vz6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ"
print(json.loads(peer_did.resolve_peer_did(testDID)))

{'id': 'did:peer:2.Ez6LSdGpdLzSijvKbyLmCf5n6o7cWi49kkUQB697hAp2GSNdu.Vz6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'authentication': [{'id': 'did:peer:2.Ez6LSdGpdLzSijvKbyLmCf5n6o7cWi49kkUQB697hAp2GSNdu.Vz6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ#6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSdGpdLzSijvKbyLmCf5n6o7cWi49kkUQB697hAp2GSNdu.Vz6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'publicKeyMultibase': 'z6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSdGpdLzSijvKbyLmCf5n6o7cWi49kkUQB697hAp2GSNdu.Vz6MkhWb8vbUhV5AZFir7B2YEvhafi7hDNp1QxyydwY9NndZn.S

In [726]:
testDID = "did:peer:2.Ez6LShB3wZEeHCf4Pe9oWsv2LSJGTpXVKsDZeFewVBmg7meLN.Vz6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ"
print(json.loads(peer_did.resolve_peer_did(testDID)))

{'id': 'did:peer:2.Ez6LShB3wZEeHCf4Pe9oWsv2LSJGTpXVKsDZeFewVBmg7meLN.Vz6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'authentication': [{'id': 'did:peer:2.Ez6LShB3wZEeHCf4Pe9oWsv2LSJGTpXVKsDZeFewVBmg7meLN.Vz6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ#6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LShB3wZEeHCf4Pe9oWsv2LSJGTpXVKsDZeFewVBmg7meLN.Vz6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovLzE5Mi4xNjguODYuNTM6ODA4MC9kaWRjb21tIiwiciI6W10sImEiOlsiZGlkY29tbS92MiJdfQ', 'publicKeyMultibase': 'z6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LShB3wZEeHCf4Pe9oWsv2LSJGTpXVKsDZeFewVBmg7meLN.Vz6MkwEA7fSRry5xdv6ioA7E6qjeZGkJNhDo4rii37SbowmQX.S